<a href="https://colab.research.google.com/github/anshupandey/Generative-AI-for-Professionals/blob/main/EY2024/C10_LangChain_Chains_Routing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U langchain-core langchain-community langgraph langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
api_key = "xxxxxxxxxxxxxxxxxxxxx"
api_version = "2024-02-01"
azure_endpoint = "https://xxxxxxxxxxx.openai.azure.com/"
model_name = "gpt-4o"

import os
os.environ["OPENAI_API_VERSION"] = api_version
os.environ["AZURE_OPENAI_ENDPOINT"] = azure_endpoint
os.environ["AZURE_OPENAI_API_KEY"] = api_key
os.environ["OPENWEATHERMAP_API_KEY"] = "29af1cea50a401d8e624eea4660b3f59"
os.environ["AZURE_OPENAI_API_VERSION"] = api_version

# How to route between sub-chains

:::info Prerequisites

This guide assumes familiarity with the following concepts:
- [LangChain Expression Language (LCEL)](/docs/concepts/#langchain-expression-language)
- [Chaining runnables](/docs/how_to/sequence/)
- [Configuring chain parameters at runtime](/docs/how_to/configure)
- [Prompt templates](/docs/concepts/#prompt-templates)
- [Chat Messages](/docs/concepts/#message-types)

:::

Routing allows you to create non-deterministic chains where the output of a previous step defines the next step. Routing can help provide structure and consistency around interactions with models by allowing you to define states and use information related to those states as context to model calls.

There are two ways to perform routing:

1. Conditionally return runnables from a [`RunnableLambda`](/docs/how_to/functions) (recommended)
2. Using a `RunnableBranch` (legacy)

We'll illustrate both methods using a two step sequence where the first step classifies an input question as being about `LangChain`, `Anthropic`, or `Other`, then routes to a corresponding prompt chain.

## Example Setup
First, let's create a chain that will identify incoming questions as being about `LangChain`, `Anthropic`, or `Other`:

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate


from langchain_openai import AzureChatOpenAI
model = AzureChatOpenAI(model=model_name,temperature=0.5)


chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `LangChain`, `Gemini`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | model
    | StrOutputParser()
)

chain.invoke({"question": "how do I call Gemini?"})

'Gemini'

In [3]:
chain.invoke({"question": "how do I use LangChain?"})

'LangChain'

In [4]:
chain.invoke({"question": "what are the models available with OpenAI API?"})

'Other'

Now, let's create three sub chains:

In [5]:
langchain_chain = PromptTemplate.from_template(
    """You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
) | model



gemini_chain = PromptTemplate.from_template(
    """You are an expert in Vertex AI Gemini Models. \
Always answer questions starting with "As Sundar Pichai told me". \
Respond to the following question:

Question: {question}
Answer:"""
) | model


general_chain = PromptTemplate.from_template(
    """Respond to the following question:

Question: {question}
Answer:"""
) | model

## Using a custom function (Recommended)

You can also use a custom function to route between different outputs. Here's an example:

In [6]:
def route(info):
    if "gemini" in info["topic"].lower():
        return gemini_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain

In [7]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    route
)

In [8]:
full_chain.invoke({"question": "how do I use Gemini?"})

AIMessage(content="As Sundar Pichai told me, to use Gemini models in Vertex AI, you first need to ensure you have access to Google Cloud and have set up Vertex AI. Once you have that in place, you can start by creating a new project or using an existing one. Here are the steps to use Gemini models:\n\n1. **Set Up Your Environment**: Install the necessary SDKs and tools. For Python, you can use the Google Cloud SDK and Vertex AI SDK.\n   \n   ```bash\n   pip install google-cloud-aiplatform\n   ```\n\n2. **Authenticate**: Make sure you authenticate your environment to access Google Cloud services.\n\n   ```python\n   from google.colab import auth\n   auth.authenticate_user()\n   ```\n\n3. **Initialize Vertex AI**: Set up your project and location.\n\n   ```python\n   from google.cloud import aiplatform\n\n   aiplatform.init(project='your-project-id', location='your-region')\n   ```\n\n4. **Create or Select a Dataset**: Use an existing dataset or create a new one for training your model.\

In [9]:
full_chain.invoke({"question": "how do I use LangChain?"})

AIMessage(content='As Harrison Chase told me, to use LangChain, you typically start by installing the necessary packages via pip. Once installed, you can begin by importing the relevant modules and setting up a language model. LangChain provides a variety of tools and utilities to facilitate the creation of language-based applications, such as text generation, question answering, and more. You can chain together different components, like prompt templates, memory, and various chains, to create complex workflows. For more detailed guidance, you might want to consult the official LangChain documentation or explore some example projects to see how everything fits together.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 118, 'prompt_tokens': 44, 'total_tokens': 162, 'completion_tokens_details': None}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_80a1bad4c7', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'f

In [10]:
full_chain.invoke({"question": "whats 2 + 2"})

AIMessage(content='2 + 2 equals 4.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 24, 'total_tokens': 32, 'completion_tokens_details': None}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_80a1bad4c7', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'seve

## Using a RunnableBranch

A `RunnableBranch` is a special type of runnable that allows you to define a set of conditions and runnables to execute based on the input. It does **not** offer anything that you can't achieve in a custom function as described above, so we recommend using a custom function instead.

A `RunnableBranch` is initialized with a list of (condition, runnable) pairs and a default runnable. It selects which branch by passing each condition the input it's invoked with. It selects the first condition to evaluate to True, and runs the corresponding runnable to that condition with the input.

If no provided conditions match, it runs the default runnable.

Here's an example of what it looks like in action:

In [11]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "gemini" in x["topic"].lower(), gemini_chain),
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    general_chain,
)
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch
full_chain.invoke({"question": "how do I use Gemini?"})

AIMessage(content="As Sundar Pichai told me, to use Gemini, you would start by accessing Google Cloud's Vertex AI platform. Gemini models are part of the advanced AI offerings within Vertex AI, designed to provide powerful machine learning capabilities. Here’s a step-by-step guide to get you started:\n\n1. **Set Up Your Environment**: Ensure you have a Google Cloud account and the necessary permissions to access Vertex AI services. Install the Google Cloud SDK and authenticate your environment.\n\n2. **Create a Vertex AI Workspace**: Navigate to the Vertex AI section in the Google Cloud Console and create a new workspace for your project.\n\n3. **Select a Gemini Model**: Within the Vertex AI interface, you can browse the available models. Choose a Gemini model that fits your use case, whether it's for natural language processing, computer vision, or another application.\n\n4. **Upload Your Data**: Prepare and upload your dataset to Google Cloud Storage. Ensure your data is properly for

In [12]:
full_chain.invoke({"question": "how do I use LangChain?"})

AIMessage(content='As Harrison Chase told me, using LangChain involves a few key steps. First, you need to install the LangChain library using pip:\n\n```bash\npip install langchain\n```\n\nOnce installed, you can start by importing the necessary modules and setting up the chain. Here\'s a simple example to get you started:\n\n```python\nfrom langchain import LangChain\n\n# Initialize the LangChain object\nchain = LangChain()\n\n# Define your prompt or input text\nprompt = "Translate the following English text to French: \'Hello, how are you?\'"\n\n# Use the chain to process the prompt\nresponse = chain.process(prompt)\n\n# Print the response\nprint(response)\n```\n\nThis is a basic example, but LangChain offers a lot of flexibility and advanced features, such as integrating with different models, customizing prompts, and handling more complex workflows. For more detailed examples and documentation, you can visit the official LangChain GitHub repository or their documentation site.', a

In [13]:
full_chain.invoke({"question": "whats 2 + 2"})

AIMessage(content='2 + 2 equals 4.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 24, 'total_tokens': 32, 'completion_tokens_details': None}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_80a1bad4c7', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'seve

## Next steps

You've now learned how to add routing to your composed LCEL chains.

Next, check out the other how-to guides on runnables in this section.